In [4]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from multielo import MultiElo, Player, Tracker
from mktools.get_data import load_data_pd
from alive_progress import alive_it
from mktools.validate_data import validate_bad_uids

pd.options.display.max_columns = 150

In [8]:
load_dotenv()

form_df = load_data_pd(sheet_name="form_data", sheet_id=os.environ["SHEET_ID"])

form_df = form_df.drop(
    columns=[x for x in form_df.columns if x.__contains__("Unnamed")]
)

form_df

,Timestamp,NEW_SESSION,MAP,PLAYERS,PLAYERS_2 [1ST],PLAYERS_2 [2ND],CHARACTERS_2 [1ST],CHARACTERS_2 [2ND],PLAYERS_3 [1ST],PLAYERS_3 [2ND],PLAYERS_3 [3RD],CHARACTERS_3 [1ST],CHARACTERS_3 [2ND],CHARACTERS_3 [3RD],PLAYERS_4 [1ST],PLAYERS_4 [2ND],PLAYERS_4 [3RD],PLAYERS_4 [4TH],CHARACTERS_4 [1ST],CHARACTERS_4 [2ND],CHARACTERS_4 [3RD],CHARACTERS_4 [4TH]
0,7/16/2024 21:20:28,NO,Wario Stadium,2,Cooper,Matt,Peach,Yoshi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7/16/2024 21:20:52,NO,Koopa Troopa Beach,3,NaN,NaN,NaN,NaN,Cooper,Matt,Blake,Peach,Yoshi,Toad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7/16/2024 21:21:14,YES,Sherbet Land,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cooper,Matt,Blake,Regan,Peach,Yoshi,Toad,Bowser
3,7/16/2024 21:24:33,NO,Yoshi Valley,3,NaN,NaN,NaN,NaN,Matt,Cooper,Blake,Yoshi,Peach,Toad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df = load_data_pd(
    sheet_name="data",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

df.tail()

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
21651,6291,736,Cole,Peach,Sherbet Land,3,4,2024-07-16,11
21652,6291,736,Triston,Bowser,Sherbet Land,4,4,2024-07-16,11
21653,6292,736,Cooper,Toad,Wario Stadium,1,3,2024-07-16,11
21654,6292,736,Connor,Peach,Wario Stadium,2,3,2024-07-16,11
21655,6292,736,Konnor,Yoshi,Wario Stadium,3,3,2024-07-16,11


In [68]:
last_uid = df.tail(1)["UID"].values[0]
last_suid = df.tail(1)["SUID"].values[0]

In [75]:
for i in range(last_uid + 1, (last_uid + form_df.shape[0] + 1), 1):
    print(i)

6293
6294
6295
6296


In [78]:
l = []

for s in form_df["NEW_SESSION"]:
    if s == "NO":
        l.append(last_suid)
    if s == "YES":
        last_suid += 1
        l.append(last_suid)

In [79]:
l

[736, 736, 737, 737]

In [80]:
form_df["UID"] = range(last_uid + 1, (last_uid + form_df.shape[0] + 1), 1)

form_df["SUID"] = l

In [81]:
form_df

,Timestamp,NEW_SESSION,MAP,PLAYERS,PLAYERS_2 [1ST],PLAYERS_2 [2ND],CHARACTERS_2 [1ST],CHARACTERS_2 [2ND],PLAYERS_3 [1ST],PLAYERS_3 [2ND],PLAYERS_3 [3RD],CHARACTERS_3 [1ST],CHARACTERS_3 [2ND],CHARACTERS_3 [3RD],PLAYERS_4 [1ST],PLAYERS_4 [2ND],PLAYERS_4 [3RD],PLAYERS_4 [4TH],CHARACTERS_4 [1ST],CHARACTERS_4 [2ND],CHARACTERS_4 [3RD],CHARACTERS_4 [4TH],UID,SUID
0,7/16/2024 21:20:28,NO,Wario Stadium,2,Cooper,Matt,Peach,Yoshi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6293,736
1,7/16/2024 21:20:52,NO,Koopa Troopa Beach,3,NaN,NaN,NaN,NaN,Cooper,Matt,Blake,Peach,Yoshi,Toad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6294,736
2,7/16/2024 21:21:14,YES,Sherbet Land,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cooper,Matt,Blake,Regan,Peach,Yoshi,Toad,Bowser,6295,737
3,7/16/2024 21:24:33,NO,Yoshi Valley,3,NaN,NaN,NaN,NaN,Matt,Cooper,Blake,Yoshi,Peach,Toad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6296,737


In [82]:
two_p_df = form_df[form_df["PLAYERS"] == 2][
    [
        "UID",
        "SUID",
        "Timestamp",
        "NEW_SESSION",
        "MAP",
        "PLAYERS",
        *[x for x in form_df.columns if x.__contains__("_2")],
    ]
]

three_p_df = form_df[form_df["PLAYERS"] == 3][
    [
        "UID",
        "SUID",
        "Timestamp",
        "NEW_SESSION",
        "MAP",
        "PLAYERS",
        *[x for x in form_df.columns if x.__contains__("_3")],
    ]
]

four_p_df = form_df[form_df["PLAYERS"] == 4][
    [
        "UID",
        "SUID",
        "Timestamp",
        "NEW_SESSION",
        "MAP",
        "PLAYERS",
        *[x for x in form_df.columns if x.__contains__("_4")],
    ]
]

two_p_df

,UID,SUID,Timestamp,NEW_SESSION,MAP,PLAYERS,PLAYERS_2 [1ST],PLAYERS_2 [2ND],CHARACTERS_2 [1ST],CHARACTERS_2 [2ND]
0,6293,736,7/16/2024 21:20:28,NO,Wario Stadium,2,Cooper,Matt,Peach,Yoshi


In [83]:
t = three_p_df.rename(
    columns={
        "PLAYERS_3 [1ST]": "NAME_1",
        "PLAYERS_3 [2ND]": "NAME_2",
        "PLAYERS_3 [3RD]": "NAME_3",
        "CHARACTERS_3 [1ST]": "CHARACTER_1",
        "CHARACTERS_3 [2ND]": "CHARACTER_2",
        "CHARACTERS_3 [3RD]": "CHARACTER_3",
    }
)

t

,UID,SUID,Timestamp,NEW_SESSION,MAP,PLAYERS,NAME_1,NAME_2,NAME_3,CHARACTER_1,CHARACTER_2,CHARACTER_3
1,6294,736,7/16/2024 21:20:52,NO,Koopa Troopa Beach,3,Cooper,Matt,Blake,Peach,Yoshi,Toad
3,6296,737,7/16/2024 21:24:33,NO,Yoshi Valley,3,Matt,Cooper,Blake,Yoshi,Peach,Toad


In [84]:
pd.wide_to_long(
    df=t,
    stubnames=["CHARACTER", "NAME"],
    i=["UID", "SUID", "Timestamp", "MAP"],
    j="PLACE",
    sep="_",
).reset_index()

,UID,SUID,Timestamp,MAP,PLACE,NEW_SESSION,PLAYERS,CHARACTER,NAME
0,6294,736,7/16/2024 21:20:52,Koopa Troopa Beach,1,NO,3,Peach,Cooper
1,6294,736,7/16/2024 21:20:52,Koopa Troopa Beach,2,NO,3,Yoshi,Matt
2,6294,736,7/16/2024 21:20:52,Koopa Troopa Beach,3,NO,3,Toad,Blake
3,6296,737,7/16/2024 21:24:33,Yoshi Valley,1,NO,3,Yoshi,Matt
4,6296,737,7/16/2024 21:24:33,Yoshi Valley,2,NO,3,Peach,Cooper
5,6296,737,7/16/2024 21:24:33,Yoshi Valley,3,NO,3,Toad,Blake
